In [23]:
pip install openai --upgrade

Note: you may need to restart the kernel to use updated packages.


In [24]:
import os
api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI

In [25]:
import json
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [26]:
client = OpenAI()
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a helpful personal math tutor, Answer questions and show your work.",
    model="gpt-4-1106-preview"
)
show_json(assistant)


{'id': 'asst_dwmMk99NSZ8nXfz8ttBZVsF3',
 'created_at': 1702739530,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a helpful personal math tutor, Answer questions and show your work.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

In [27]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_38nTOZfK8eiaSqIrpm8VUpIO',
 'created_at': 1702739530,
 'metadata': {},
 'object': 'thread'}

In [28]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve this problem: 3x + 11 = 14, can you help me?"
)
show_json(message) # 此时还没有真正请求 OpenAI 进行处理

{'id': 'msg_wIYikjg3ccJk2JbOoZ3h2S34',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve this problem: 3x + 11 = 14, can you help me?'},
   'type': 'text'}],
 'created_at': 1702739531,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_38nTOZfK8eiaSqIrpm8VUpIO'}

In [29]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)
show_json(run)
# 从输出可以看到 status 是 queued，说明 run 是异步执行的，我们需要写个方法来等待执行完成

{'id': 'run_35E0dk0jed3RzDRjikbeiYu4',
 'assistant_id': 'asst_dwmMk99NSZ8nXfz8ttBZVsF3',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1702739531,
 'expires_at': 1702740131,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a helpful personal math tutor, Answer questions and show your work.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_38nTOZfK8eiaSqIrpm8VUpIO',
 'tools': []}

In [30]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        time.sleep(0.5)
    return run

In [31]:
run = wait_on_run(run, thread)
show_json(run) # 此时可以看到 status 是 completed，说明运行完成

{'id': 'run_35E0dk0jed3RzDRjikbeiYu4',
 'assistant_id': 'asst_dwmMk99NSZ8nXfz8ttBZVsF3',
 'cancelled_at': None,
 'completed_at': 1702739540,
 'created_at': 1702739531,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a helpful personal math tutor, Answer questions and show your work.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1702739532,
 'status': 'completed',
 'thread_id': 'thread_38nTOZfK8eiaSqIrpm8VUpIO',
 'tools': []}

In [32]:
message = client.beta.threads.messages.list(thread_id=thread.id)
show_json(message) # 打印结果

{'data': [{'id': 'msg_EaIPRHTVuT7MLJbnm4XxH6Ae',
   'assistant_id': 'asst_dwmMk99NSZ8nXfz8ttBZVsF3',
   'content': [{'text': {'annotations': [],
      'value': "Certainly! To solve for x in the equation 3x + 11 = 14, we want to isolate x on one side of the equation. Here's how we can do that step by step:\n\n1. Subtract 11 from both sides of the equation to get the term with x by itself on the left side:\n\n3x + 11 - 11 = 14 - 11\n\nThe 11s on the left cancel out, leaving us with:\n\n3x = 3\n\n2. Now, to solve for x, divide both sides of the equation by 3, which is the coefficient of x:\n\n3x / 3 = 3 / 3\n\nx = 1\n\nSo the solution to the equation is x = 1."},
     'type': 'text'}],
   'created_at': 1702739533,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_35E0dk0jed3RzDRjikbeiYu4',
   'thread_id': 'thread_38nTOZfK8eiaSqIrpm8VUpIO'},
  {'id': 'msg_wIYikjg3ccJk2JbOoZ3h2S34',
   'assistant_id': None,
   'content': [{'text':

In [35]:
# 连续对话
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you explain this to me?"
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

wait_on_run(run, thread)
message = client.beta.threads.messages.list(thread_id=thread.id, order="asc", after=message.id)
show_json(message)

{'data': [{'id': 'msg_Qwm0JIi08GNx0O1gJn6EC3HO',
   'assistant_id': 'asst_dwmMk99NSZ8nXfz8ttBZVsF3',
   'content': [{'text': {'annotations': [],
      'value': 'Certainly! Elasticsearch is a search engine based on the Lucene library and it\'s designed to handle a variety of use-cases that involve the search and analysis of large amounts of data. Instead of a "query cache" similar to those found in traditional SQL databases, Elasticsearch uses a combination of different caching mechanisms to speed up searches, but it does not cache the actual results of queries in the same way a regular database might cache results of select statements.\n\nLet me explain the types of caches in Elasticsearch that contribute to improving search performance:\n\n1. **Node Query Cache**: Elasticsearch caches the filter part of search requests. Filters are deterministic, meaning that given a set of data and a filter query, the result is always going to be the same. Elasticsearch takes advantage of this to cac